## Assignment #2

* Release date: 2021.04.12 Mon
* Due date: **2021.04.18 Sun 23:59** (will not accept late submission)
* Submission format: notebook file which can be executed in Colab environment
* Weighting: 5% (total 50 pts)
* You will build a multi-class classification model using Reuters dataset.

> ### Loading and preprocessing data

In [123]:
from tensorflow.keras.datasets import reuters

# Like IMDB, the argument num_words restricts the data to 
# the 10,000 most frequently occurring words 
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

In [124]:
# decoding newswires back to text
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i-3, '?') for i in train_data[0]])

* (10pts) Write the codes for preprocessing data
  * For inputs, the data we have should be converted to binary vectors.
  * For labels, determine an appropriate format by referring to the arguments of model.compile function below.

In [126]:
# write preprocessing codes
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [129]:
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results
    
one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

In [110]:
# from tensorflow.keras.utils import to_categorical

# one_hot_train_labels = to_categorical(train_labels)
# one_hot_test_labels = to_categorical(test_labels)

> ### Building the network

In [132]:
# Do not modify this block
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

In [136]:
# Do not modify this block
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

> ### Validation

* We employ *k-fold cross validation* as validation method of our model.
* **(15pts)** Write a code in the below to perform *10-fold cross validation*.
* **For each fold, save a model at every epoch in your Google Drive.**

In [137]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

In [139]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512)

results = model.evaluate(x_test, one_hot_test_labels)

Train on 7982 samples
Epoch 1/20
7982/7982 [==============================] - 0s 41us/sample - loss: 0.1076 - accuracy: 0.9578
Epoch 2/20
7982/7982 [==============================] - 0s 36us/sample - loss: 0.1034 - accuracy: 0.9592
Epoch 3/20
7982/7982 [==============================] - 0s 34us/sample - loss: 0.1064 - accuracy: 0.9590
Epoch 4/20
7982/7982 [==============================] - 0s 37us/sample - loss: 0.1039 - accuracy: 0.9572
Epoch 5/20
7982/7982 [==============================] - 0s 37us/sample - loss: 0.0999 - accuracy: 0.9577
Epoch 6/20
7982/7982 [==============================] - 0s 36us/sample - loss: 0.0978 - accuracy: 0.9578
Epoch 7/20
7982/7982 [==============================] - 0s 35us/sample - loss: 0.1011 - accuracy: 0.9587
Epoch 8/20
7982/7982 [==============================] - 0s 35us/sample - loss: 0.0938 - accuracy: 0.9592
Epoch 9/20
7982/7982 [==============================] - 0s 35us/sample - loss: 0.0986 - accuracy: 0.9575
Epoch 10/20
7982/7982 [==========

In [140]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
%cd /content/gdrive

In [ ]:
!ls

In [ ]:
%cd 'My Drive'/exp

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = '/content/gdrive/My Drive/exp/model.{epoch:02d}.hdf5'
modelckpt = ModelCheckpoint(filepath=filepath)

model.fit(partial_x_train,
          partial_y_train,
          epochs=9,
          batch_size=512,
          validation_data(x_val, y_val),
          callbacks=[modelckpt])

results = model.evaluate(x_test, one_hot_test_labels)

In [ ]:
# write a code for 10-fold cross validation here

k = 10
num_validation_samples = len(x_test) // k

np.random.shuffle(x_test)

validation_scores = []
for fold in range(k):
  validation_data = x_test[num_validation_samples*fold : num_validation_samples*(fold+1)]
  training_data = x_test[:num_validation_samples*fold] + x_test[num_validation_samples*(fold+1):]

  model = get_model()
  model.train(training_data)
  validation_score = model.evaluate(validation_data)
  validation_scores.append(validation_score)

validation_score = np.average(validation_scores)

model = get_model()
model.train(x_test)
test_score = model.evaluate(x_test)

* **(10pts)** Plotting the training and validation accuracy
  * To obtain the validation accuracy at the end of every epoch, just average the performances of all folds.

In [117]:
# write a code for plotting the training and validation accuracy

import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Traning and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined

> ### (15pts) Inference

* Find the best performance model by seeing the performance plot.
* Calculate the accuracy on test set using the best performance model.
  * Here, you should use a majority voting method to get the prediction for a test data point.
  * Specifically, given a test data point, get the predicted class from the trained model on each fold, and then decide the final predicted class by a majority voting.
* **Do not retrain the model.**

In [ ]:
# write a code here